# Deep Q-Network implementation

This notebook shamelessly demands you to implement a DQN - an approximate q-learning algorithm with experience replay and target networks - and see if it works any better this way.

In [ ]:
#XVFB will be launched if you run on a server
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

__Frameworks__ - we'll accept this homework in any deep learning framework. This particular notebook was designed for tensorflow, but you will find it easy to adapt it to almost any python-based deep learning framework.

In [ ]:
import gym
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

In [ ]:
train_from_scratch = False

### Let's play some old videogames
![img](https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/nerd.png)

This time we're gonna apply approximate q-learning to an atari game called Breakout. It's not the hardest thing out there, but it's definitely way more complex than anything we tried before.


### Processing game image 

Raw atari images are large, 210x160x3 by default. However, we don't need that level of detail in order to learn them.

We can thus save a lot of time by preprocessing game image, including
* Resizing to a smaller shape, 64 x 64
* Converting to grayscale
* Cropping irrelevant image parts (top & bottom)

In [ ]:
# Install scikit image if not already installed
!pip install scikit-image

In [ ]:
# Seems like this is not installed
!pip install atari-py

In [ ]:
from gym.core import ObservationWrapper
from gym.spaces import Box
from skimage.color import rgb2gray
from skimage.transform import resize


class PreprocessAtari(ObservationWrapper):
    """
    Applies a preprocesser to the environment
    """
    
    def __init__(self, env):
        """
        A gym wrapper that crops, scales image into the desired shapes and optionally grayscales it.
        
        Parameters
        ----------
        env : gym-object
            The environment to play with
        """
        ObservationWrapper.__init__(self, env)
        
        self.img_size = (64, 64)
        # NOTE: Edited for consistency
        #       https://www.coursera.org/learn/practical-rl/programming/upglq/dqn-breakout/discussions/threads/yzC8W14LEei7pAoHCSt0dA?sort=createdAtAsc&page=1
        self.observation_space = \
            Box(low=0.0, high=1.0, shape=(self.img_size[0], self.img_size[1], 1), dtype=np.float32)

    def observation(self, img):
        """
        Returns the observation given a raw image
        
        This function will:
            - Center crop the image (160, 160, 3)
            - Resize the image
            - Cast the image to grayscale
            - Normalize the pixels in the (0, 1) range
        
        Parameters
        ----------
        img : np.array, shape (210, 160, 3)
            The raw input image
        
        Returns
        -------
        proc_img : np.array, shape (*self.img_size)
            The processed image
        """
        
        # NOTE: After trial and error we cropped the image so that it just
        #       include the boarders
        # NOTE: The subsequent resize will slightly stretch the image horizontally
        proc_img = img[30:-15, 6:-6]
        # Resizing
        # NOTE: This recasts to 0, 1
        #       https://www.coursera.org/learn/practical-rl/programming/upglq/dqn-breakout/discussions/threads/_JqWyHn4Eeiv7hJZeB6Vjg
        proc_img = resize(proc_img, self.img_size, mode='reflect', anti_aliasing=True)
        # NOTE: Ehancing red color (the color of the ball) by shifting the relative distribtion
        proc_img[..., 0] *= 1.2
        proc_img[..., 1] *= 0.3
        proc_img[..., 2] *= 0.2
        # Grayscaling
        proc_img = rgb2gray(proc_img)
        # Sacling (based on max pixel) and cast to float32
        proc_img = proc_img.astype(np.float32)/0.2977867

        # Make sure the dimension is 3
        proc_img = proc_img[..., np.newaxis]
              
        return proc_img

In [ ]:
env = gym.make("BreakoutDeterministic-v0")
env.reset()
plt.title("The original game image")
plt.imshow(env.render("rgb_array"))

In [ ]:
# Spawn game instance for tests
env = gym.make("BreakoutDeterministic-v0") #create raw env
env = PreprocessAtari(env)

observation_shape = env.observation_space.shape
n_actions = env.action_space.n

obs = env.reset()

# test observation
assert obs.ndim == 3, "observation must be [batch, time, channels] even if there's just one channel"
# assert obs.shape == observation_shape
assert obs.dtype == 'float32'
assert len(np.unique(obs))>2, "your image must not be binary"
assert 0 <= np.min(obs) and np.max(obs) <=1, "convert image pixels to (0,1) range"

print("Formal tests seem fine. Here's an example of what you'll get.")

fig, ax = plt.subplots()
fig.suptitle("what your network gonna see")
# NOTE: Edit as we have something with ndim = 3
im = ax.imshow(obs[..., 0], interpolation='none', cmap='gray')
_ = fig.colorbar(im, ax=ax)

In [ ]:
obs = env.reset()
# NOTE: Looks like ball starts after taking action 1
obs, _, _, _ = env.step(1)

fig, ax = plt.subplots()
fig.suptitle("After on step of action 1")
# NOTE: Edit as we have something with ndim = 3
im = ax.imshow(obs[..., 0], interpolation='none', cmap='gray')
_ = fig.colorbar(im, ax=ax)

### Frame buffer

Our agent can only process one observation at a time, so we gotta make sure it contains enough information to fing optimal actions. For instance, agent has to react to moving objects so he must be able to measure object's velocity.

To do so, we introduce a buffer that stores 4 last images. This time everything is pre-implemented for you.

In [ ]:
from framebuffer import FrameBuffer

def make_env():
    """
    Make the environment with such that the last 4 images are buffered in the image
    """
    
    # NOTE: v4 is the fourth sematic version of break out
    #       https://github.com/openai/baselines/issues/552
    env = gym.make("BreakoutDeterministic-v4")
    env = PreprocessAtari(env)
    env = FrameBuffer(env, n_frames=4, dim_order='tensorflow')
    return env

env = make_env()
env.reset()
n_actions = env.action_space.n
state_dim = env.observation_space.shape

In [ ]:
for _ in range(50):
    obs, _, _, _ = env.step(env.action_space.sample())

plt.title("Game image")
plt.imshow(env.render("rgb_array"))
plt.show()
plt.title("Agent observation (4 frames left to right)")
plt.imshow(obs.transpose([0,2,1]).reshape([state_dim[0], -1]));

### Building a network

We now need to build a neural network that can map images to state q-values. This network will be called on every agent's step so it better not be resnet-152 unless you have an array of GPUs. Instead, you can use strided convolutions with a small number of features to save time and memory.

You can build any architecture you want, but for reference, here's something that will more or less work:

![img](https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/dqn_arch.png)

In [ ]:
import tensorflow as tf
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [ ]:
import keras.layers as L
import keras

class DQNAgent(object):
    """
    The deep q-network agent
    """
    
    def __init__(self, name, state_shape, n_actions, epsilon=0, reuse=False):
        """
        The constructor to the DQN
        
        Creates the network
        
        Parameters
        ----------
        name : str
            The name of the variable scope
        state_shape : tuple
            Shape of the state
            For example (64, 64, 4)
        n_actions : int
            The number of allowed actions
        epsilon : float
            The epsilon used for learning
        reuse : bool
            Whether or not the variable scope should be reused
        """
        
        with tf.variable_scope(name, reuse=reuse):
            
            # Setup the network
            self.network = keras.models.Sequential()
            self.network.add(L.InputLayer(state_shape))
            self.network.add(L.Conv2D(filters=16, 
                                      kernel_size=(3, 3),
                                      strides=(2, 2)))
            self.network.add(L.ReLU())
            self.network.add(L.Conv2D(filters=32, 
                                      kernel_size=(3, 3),
                                      strides=(2, 2)))
            self.network.add(L.ReLU())
            self.network.add(L.Conv2D(filters=32, 
                                      kernel_size=(3, 3),
                                      strides=(2, 2)))
            self.network.add(L.Flatten())
            self.network.add(L.Dense(256))
            self.network.add(L.ReLU())
            self.network.add(L.Dense(n_actions))
            
            # Prepare a graph for agent step
            self.state_t = tf.placeholder('float32', [None,] + list(state_shape))
            self.qvalues_t = self.get_symbolic_qvalues(self.state_t)
            
        self.weights = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=name)
        self.epsilon = epsilon

    def get_symbolic_qvalues(self, state_t):
        """
        Takes the agent's observation and returns its (symbolic) qvalues
        
        Parameters
        ----------
        state_t : tf.Tensor, shape (batch_nr, *state_shape)
            The current state
        
        Returns
        -------
        qvalues : tf.Tensor, shape (batch_nr, n_actions)
            The corresponding Q-values
        """
        
        qvalues = self.network(state_t)
        
        assert tf.is_numeric_tensor(qvalues) and qvalues.shape.ndims == 2, \
            "please return 2d tf tensor of qvalues [you got %s]" % repr(qvalues)
        assert int(qvalues.shape[1]) == n_actions
        
        return qvalues
    
    def get_qvalues(self, state_t):
        """
        Same as get_symbolic_qvalues step except it operates on numpy arrays
        
        Parameters
        ----------
        state_t : np.array, shape (batch_nr, *state_shape)
            The current state
        
        Returns
        -------
        qvalues : np.array, shape (batch_nr, n_actions)
            The corresponding Q-values
        """
        
        sess = tf.get_default_session()
        return sess.run(self.qvalues_t, {self.state_t: state_t})
    
    def sample_actions(self, qvalues):
        """
        Pick actions given qvalues.
        
        Uses epsilon-greedy exploration strategy.
        
        Parameters
        ----------
        qvalues : np.array, shape (batch_nr, n_actions)
            The Q-values to sample from
        
        Returns
        -------
        int
            The action
        """
        
        epsilon = self.epsilon
        batch_size, n_actions = qvalues.shape
        random_actions = np.random.choice(n_actions, size=batch_size)
        best_actions = qvalues.argmax(axis=-1)
        should_explore = np.random.choice([0, 1], batch_size, p = [1-epsilon, epsilon])
        
        return np.where(should_explore, random_actions, best_actions)

In [ ]:
agent = DQNAgent("dqn_agent", state_dim, n_actions, epsilon=0.5)
sess.run(tf.global_variables_initializer())

Now let's try out our agent to see if it raises any errors.

In [ ]:
def evaluate(env, agent, n_games=1, greedy=False, t_max=10000):
    """
    Plays `n_games` games
    
    The game will finish either when the game has ended or when `t_max` is reached
    
    Parameters
    ----------
    env : gym-object
        The environment to play with
    agent : DQNAgent
        The agent to play with
    n_games : int
        Number of games to play
    greedy : bool
        If greedy, the agent picks actions as argmax(qvalues)
    t_max : int
        The maximum number of states before terminating the game
        
    Returns
    -------
    float
        The mean rewards
    """
    
    i=0
    
    rewards = []
    for _ in range(n_games):
        s = env.reset()
        reward = 0
        # NOTE: Changed t_max with while True
        #       https://github.com/openai/gym/issues/430
        while True:
            qvalues = agent.get_qvalues([s])
            action = qvalues.argmax(axis=-1)[0] if greedy else agent.sample_actions(qvalues)[0]
            s, r, done, _ = env.step(action)
            reward += r
            i+=1
            # NOTE: It seems like the environment doesn't set done appropriately
            if done or i > t_max: 
                break
                
        rewards.append(reward)
    return np.mean(rewards)

In [ ]:
evaluate(env, agent, n_games=1)

### Experience replay
For this assignment, we provide you with experience replay buffer. If you implemented experience replay buffer in last week's assignment, you can copy-paste it here __to get 2 bonus points__.

![img](https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/exp_replay.png)

#### The interface is fairly simple:
* `exp_replay.add(obs, act, rw, next_obs, done)` - saves (s,a,r,s',done) tuple into the buffer
* `exp_replay.sample(batch_size)` - returns observations, actions, rewards, next_observations and is_done for `batch_size` random samples.
* `len(exp_replay)` - returns number of elements stored in replay buffer.

In [ ]:
from replay_buffer import ReplayBuffer
exp_replay = ReplayBuffer(10)

for _ in range(30):
    exp_replay.add(env.reset(), env.action_space.sample(), 1.0, env.reset(), done=False)

obs_batch, act_batch, reward_batch, next_obs_batch, is_done_batch = exp_replay.sample(5)

assert len(exp_replay) == 10, "experience replay size should be 10 because that's what maximum capacity is"

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
def play_and_record(agent, env, exp_replay, n_steps=1, progressbar=True):
    """
    Play the game for exactly n steps, record every (s, a, r, s', done) to replay buffer. 
    Whenever game ends, add record with done=True and reset the game.
    
    Notes
    -----
    Please do not env.reset() unless env is done.
    It is guaranteed that env has done=False when passed to this function.
    
    Parameters
    ----------
    env : gym-object
        The environment to play with
    agent : DQNAgent
        The agent to play with
    exp_replay : ReplayBuffer-like
        The replay buffer to use
    n_steps : int
        The number of steps to use
    progressbar : bool
        Whether or not to use a progressbar
    
    Returns
    -------
    sum_of_rewards : float
        The sum of rewards over time
    """
    
    # State at the beginning of rollout
    s = env.framebuffer
    
    sum_of_rewards = 0
    
    # NOTE: Initialize the last_info
    last_info = None
    
    # Play the game for n_steps 
    for _ in tqdm_notebook(range(n_steps), desc='n_step', disable=not(progressbar)):
        qvalues = agent.get_qvalues([s])
        # Non-greedy choice of action
        action = agent.sample_actions(qvalues)[0]
        new_s, r, done, info = env.step(action)
        # NOTE: We penalize the agent if it died last time
        r = -10 if (last_info is not None 
                    and last_info['ale.lives'] > info['ale.lives']) \
                else r
        last_info = info
        
        sum_of_rewards += r
        
        # Add to the replay buffer
        exp_replay.add(obs_t=s, action=action, reward=r, obs_tp1=new_s, done=done)
        if done:
            new_s = env.reset()
        
        # Update state
        s = new_s
        
    return sum_of_rewards

In [ ]:
# testing your code. This may take a minute...
exp_replay = ReplayBuffer(20000)

play_and_record(agent, env, exp_replay, n_steps=10000)

# if you're using your own experience replay buffer, some of those tests may need correction. 
# just make sure you know what your code does
assert len(exp_replay) == 10000, "play_and_record should have added exactly 10000 steps, "\
                                 "but instead added %i"%len(exp_replay)
is_dones = list(zip(*exp_replay._storage))[-1]

assert 0 < np.mean(is_dones) < 0.1, "Please make sure you restart the game whenever it is 'done' and record the is_done correctly into the buffer."\
                                    "Got %f is_done rate over %i steps. [If you think it's your tough luck, just re-run the test]"%(np.mean(is_dones), len(exp_replay))
    
for _ in range(100):
    obs_batch, act_batch, reward_batch, next_obs_batch, is_done_batch = exp_replay.sample(10)
    assert obs_batch.shape == next_obs_batch.shape == (10,) + state_dim
    assert act_batch.shape == (10,), "actions batch should have shape (10,) but is instead %s"%str(act_batch.shape)
    assert reward_batch.shape == (10,), "rewards batch should have shape (10,) but is instead %s"%str(reward_batch.shape)
    assert is_done_batch.shape == (10,), "is_done batch should have shape (10,) but is instead %s"%str(is_done_batch.shape)
    assert [int(i) in (0,1) for i in is_dones], "is_done should be strictly True or False"
    assert [0 <= a <= n_actions for a in act_batch], "actions should be within [0, n_actions]"
    
print("Well done!")

### Target networks

We also employ the so called "target network" - a copy of neural network weights to be used for reference Q-values:

The network itself is an exact copy of agent network, but it's parameters are not trained. Instead, they are moved here from agent's actual network every so often.

$$ Q_{reference}(s,a) = r + \gamma \cdot \max _{a'} Q_{target}(s',a') $$

![img](https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/target_net.png)



In [ ]:
target_network = DQNAgent("target_network", state_dim, n_actions)

In [ ]:
def load_weights_into_target_network(agent, target_network):
    """ 
    Assign target_network.weights variables to their respective agent.weights values
    
    Parameters
    ----------
    agent : DQNAgent
        The agent to copy weights from
    target_network : DQNAgent
        The agent to copy weights to
        
    Returns
    -------
    assigns : list
        A list of assignment from the agent weights to the target weights
    """
    
    assigns = []
    for w_agent, w_target in zip(agent.weights, target_network.weights):
        assigns.append(tf.assign(w_target, w_agent, validate_shape=True))
    return assigns

In [ ]:
# Create the tf copy graph only once
copy_step = load_weights_into_target_network(agent, target_network)
sess.run(copy_step)
# Check that it works
sess.run([tf.assert_equal(w, w_target)
          for w, w_target in zip(agent.weights, target_network.weights)])
print("It works!")

### Learning with... Q-learning
Here we write a function similar to `agent.update` from tabular q-learning.

In [ ]:
# placeholders that will be fed with exp_replay.sample(batch_size)
obs_ph = tf.placeholder(tf.float32, shape=(None,) + state_dim)
actions_ph = tf.placeholder(tf.int32, shape=[None])
rewards_ph = tf.placeholder(tf.float32, shape=[None])
next_obs_ph = tf.placeholder(tf.float32, shape=(None,) + state_dim)
is_done_ph = tf.placeholder(tf.float32, shape=[None])

is_not_done = 1 - is_done_ph
gamma = 0.99

Take q-values for actions agent just took

In [ ]:
current_qvalues = agent.get_symbolic_qvalues(obs_ph)
current_action_qvalues = tf.reduce_sum(tf.one_hot(actions_ph, n_actions) * current_qvalues, axis=1)

Compute Q-learning TD error:

$$ L = { 1 \over N} \sum_i [ Q_{\theta}(s,a) - Q_{reference}(s,a) ] ^2 $$

With Q-reference defined as

$$ Q_{reference}(s,a) = r(s,a) + \gamma \cdot max_{a'} Q_{target}(s', a') $$

Where
* $Q_{target}(s',a')$ denotes q-value of next state and next action predicted by __target_network__
* $s, a, r, s'$ are current state, action, reward and next state respectively
* $\gamma$ is a discount factor defined two cells above.

In [ ]:
# Compute q-values for NEXT states with target network
next_qvalues_target = target_network.get_symbolic_qvalues(next_obs_ph)

# Compute state values by taking max over next_qvalues_target for all actions
# NOTE: Multiply with is_not_done as the next state value will be zero if the game is done 
next_state_values_target = tf.reduce_max(next_qvalues_target, axis=-1) * is_not_done

# Compute Q_reference(s,a) as per formula above.
reference_qvalues = rewards_ph + gamma*next_state_values_target

# Define loss function for sgd.
td_loss = (current_action_qvalues - reference_qvalues) ** 2
td_loss = tf.reduce_mean(td_loss)

train_step = tf.train.AdamOptimizer(1e-3).minimize(td_loss, var_list=agent.weights)

In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
for chk_grad in tf.gradients(reference_qvalues, agent.weights):
    error_msg = "Reference q-values should have no gradient w.r.t. agent weights. Make sure you used target_network qvalues! "
    error_msg += "If you know what you're doing, ignore this assert."
    assert chk_grad is None or np.allclose(sess.run(chk_grad), sess.run(chk_grad * 0)), error_msg

assert tf.gradients(reference_qvalues, is_not_done)[0] is not None, "make sure you used is_not_done"
assert tf.gradients(reference_qvalues, rewards_ph)[0] is not None, "make sure you used rewards"
assert tf.gradients(reference_qvalues, next_obs_ph)[0] is not None, "make sure you used next states"
assert tf.gradients(reference_qvalues, obs_ph)[0] is None, "reference qvalues shouldn't depend on current observation!" # ignore if you're certain it's ok
print("Splendid!")

### Main loop

It's time to put everything together and see if it learns anything.

In [ ]:
from tqdm import trange
from IPython.display import clear_output
import matplotlib.pyplot as plt
from pandas import DataFrame
moving_average = lambda x, span, **kw: DataFrame({'x':np.asarray(x)}).x.ewm(span=span, **kw).mean().values
%matplotlib inline

mean_rw_history = []
td_loss_history = []

In [ ]:
exp_replay = ReplayBuffer(10**5)
play_and_record(agent, env, exp_replay, n_steps=10000)

def sample_batch(exp_replay, batch_size):
    obs_batch, act_batch, reward_batch, next_obs_batch, is_done_batch = exp_replay.sample(batch_size)
    return {
        obs_ph:obs_batch, actions_ph:act_batch, rewards_ph:reward_batch, 
        next_obs_ph:next_obs_batch, is_done_ph:is_done_batch
    }

In [ ]:
data_dir = Path('data').absolute()
if not data_dir.is_dir():
    data_dir.mkdir(parents=True, exist_ok=True)
    
model_path = data_dir.joinpath('model.ckpt')
mean_rw_history_path = data_dir.joinpath('mean_rw_history.pkl')
td_loss_history_path = data_dir.joinpath('td_loss_history.pkl')
epsilon_path = data_dir.joinpath('epsilon.pkl')
exp_replay_path = data_dir.joinpath('replay_buffer.pkl')
paths = [model_path, 
         mean_rw_history_path,
         td_loss_history_path,
         epsilon_path,
         exp_replay_path]

In [ ]:
def plot_progress(mean_rw_history, td_loss_history):
    """
    Plots the progress
    
    Parameters
    ----------
    mean_rw_history : list
        List of the mean reward
    td_loss_history : list
        List of the TD loss
    """
        
    plt.figure(figsize=[48, 4])
    plt.subplot(1,2,1)
    plt.title("mean reward per game")
    plt.plot(mean_rw_history)
    plt.grid()
    
    plt.figure(figsize=[48, 4])
    plt.subplot(1,2,2)
    plt.title("TD loss history (moving average)")
    plt.plot(moving_average(np.array(td_loss_history), span=100, min_periods=100))
    plt.grid()
    plt.show()

In [ ]:
def save_variables(sess, paths, variables):
    """
    Save the variables
    
    Parameters
    ----------
    sess : tf.Session
        The current session
    paths : list
        The paths as either Path or str in the following order:
        - model_path
        - mean_rw_history_path
        - td_loss_history_path
        - epsilon_path
        - exp_replay_path
    variables : list
        List of the variables in the following order:
        - mean_rw_history
        - td_loss_history
        - agent (contains the epsilon)
        - exp_replay
    """
    
    saver = tf.train.Saver()
    
    saver.save(sess, str(model_path))
    print('Saved to {}'.format(model_path))
    
    with mean_rw_history_path.open('wb') as f:
        pickle.dump(mean_rw_history, f, pickle.HIGHEST_PROTOCOL)
        print('Saved to {}'.format(mean_rw_history_path))
    
    with td_loss_history_path.open('wb') as f:
        pickle.dump(td_loss_history, f, pickle.HIGHEST_PROTOCOL)
        print('Saved to {}'.format(td_loss_history_path))
        
    with epsilon_path.open('wb') as f:
        pickle.dump(agent.epsilon, f, pickle.HIGHEST_PROTOCOL)
        print('Saved to {}'.format(epsilon_path))
    
    with exp_replay_path.open('wb') as f:
        pickle.dump(exp_replay, f, pickle.HIGHEST_PROTOCOL)
        print('Saved to {}'.format(exp_replay_path))

In [ ]:
def load_variables(sess, paths, agent):
    """
    Save the variables
    
    Parameters
    ----------
    sess : tf.Session
        The current session
    paths : list
        The paths as either Path or str in the following order:
        - model_path
        - mean_rw_history_path
        - td_loss_history_path
        - epsilon_path
        - exp_replay_path
    agent : DQNAgent
        The agent where the epsilon resides
        
    Returns
    -------
    tuple
        List of the variables in the following order:
        - mean_rw_history
        - td_loss_history
        - agent (contains the epsilon)
        - exp_replay
    """
    
    saver = tf.train.Saver()
    
    saver.restore(sess, str(model_path))
    print('Loaded from {}'.format(model_path))
    
    with mean_rw_history_path.open('rb') as f:
        mean_rw_history = pickle.load(f)
        print('Loaded from {}'.format(mean_rw_history_path))
        
    with td_loss_history_path.open('rb') as f:
        td_loss_history = pickle.load(f)
        print('Loaded from {}'.format(td_loss_history_path))
    
    with epsilon_path.open('rb') as f:
        agent.epsilon = pickle.load(f)
        print('Loaded from {}'.format(epsilon_path))
    
    with exp_replay_path.open('rb') as f:
        exp_replay = pickle.load(f)
        print('Loaded from {}'.format(exp_replay_path))
        
    return (sess,
            mean_rw_history,
            td_loss_history,
            agent,
            exp_replay)

In [ ]:
if not model_path.is_file() and train_from_scratch:
    for i in trange(10**6):
        # Play
        play_and_record(agent, env, exp_replay, 10, progressbar=False)
        
        # Train
        # NOTE: Doubled the batch size
        _, loss_t = sess.run([train_step, td_loss], sample_batch(exp_replay, batch_size=128))
        td_loss_history.append(loss_t)
        
        # Adjust agent parameters
        if i % 500 == 0:
            # NOTE: Bugfix
            copy_step = load_weights_into_target_network(agent, target_network) 
            sess.run(copy_step)
    
            agent.epsilon = max(agent.epsilon * 0.99, 0.01)
            mean_rw_history.append(evaluate(make_env(), agent, n_games=3))
            
            if np.mean(mean_rw_history[-10:]) > 15:
                print("That's good enough for tutorial.")
                break
        
        if i % 100 == 0:
            clear_output(True)
            print("buffer size = %i, epsilon = %.5f, mean reward = %.5f" % (len(exp_replay), 
                                                                            agent.epsilon,
                                                                            np.mean(mean_rw_history[-10:])))
            assert not np.isnan(loss_t)
            plot_progress(mean_rw_history, td_loss_history)
    
    variables = [mean_rw_history,
                 td_loss_history,
                 agent,
                 exp_replay]
    
    save_variables(sess, paths, variables)
            
else:
    sess, mean_rw_history, td_loss_history, agent, exp_replay = \
        load_variables(sess, paths, agent)
    print("buffer size = %i, epsilon = %.5f, mean reward = %.5f" % (len(exp_replay), 
                                                                    agent.epsilon,
                                                                    np.mean(mean_rw_history[-10:])))
    plot_progress(mean_rw_history, td_loss_history)

__ How to interpret plots: __


This aint no supervised learning so don't expect anything to improve monotonously. 
* __ TD loss __ is the MSE between agent's current Q-values and target Q-values. It may slowly increase or decrease, it's ok. The "not ok" behavior includes going NaN or stayng at exactly zero before agent has perfect performance.
* __ mean reward__ is the expected sum of r(s,a) agent gets over the full game session. It will oscillate, but on average it should get higher over time (after a few thousand iterations...). 
 * In basic q-learning implementation it takes 5-10k steps to "warm up" agent before it starts to get better.
* __ buffer size__ - this one is simple. It should go up and cap at max size.
* __ epsilon__ - agent's willingness to explore. If you see that agent's already at 0.01 epsilon before it's average reward is above 0 - __ it means you need to increase epsilon__. Set it back to some 0.2 - 0.5 and decrease the pace at which it goes down.
* Also please ignore first 100-200 steps of each plot - they're just oscillations because of the way moving average works.

At first your agent will lose quickly. Then it will learn to suck less and at least hit the ball a few times before it loses. Finally it will learn to actually score points.

__Training will take time.__ A lot of it actually. An optimistic estimate is to say it's gonna start winning (average reward > 10) after 10k steps. 

But hey, look on the bright side of things:

![img](https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/training.png)

### Video

In [ ]:
# NOTE: With epsilon 0 the same thing will happen every time
#       https://www.coursera.org/learn/practical-rl/programming/upglq/dqn-breakout/discussions/threads/Q7G2OoOxEeiUgAo3aYsv-A
agent.epsilon = 0 # Don't forget to reset epsilon back to previous value if you want to go on training

In [ ]:
# Record sessions
# NOTE: Explaination of monitor
#       https://hub.packtpub.com/openai-gym-environments-wrappers-and-monitors-tutorial/
import gym.wrappers
env_monitor = gym.wrappers.Monitor(make_env(), directory="videos", force=True)
# NOTE: As we use epsilon 0 and a deterministic environment, the same thing will happen over and over again. 
#       Hence we will just be needing 1 example
sessions = [evaluate(env_monitor, agent, n_games=1) for _ in tqdm_notebook(range(1))]
env_monitor.close()

In [ ]:
# Show video
from IPython.display import HTML
import os

video_names = list(filter(lambda s:s.endswith(".mp4"),os.listdir("./videos/")))

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format("./videos/"+video_names[-1])) #this may or may not be _last_ video. Try other indices

## More

If you want to play with DQN a bit more, here's a list of things you can try with it:

### Easy:
* Implementing __double q-learning__ shouldn't be a problem if you've already have target networks in place.
  * You will probably need `tf.argmax` to select best actions
  * Here's an original [article](https://arxiv.org/abs/1509.06461)

* __Dueling__ architecture is also quite straightforward if you have standard DQN.
  * You will need to change network architecture, namely the q-values layer
  * It must now contain two heads: V(s) and A(s,a), both dense layers
  * You should then add them up via elemwise sum layer.
  * Here's an [article](https://arxiv.org/pdf/1511.06581.pdf)

### Hard: Prioritized experience replay

In this section, you're invited to implement prioritized experience replay

* You will probably need to provide a custom data structure
* Once pool.update is called, collect the pool.experience_replay.observations, actions, rewards and is_alive and store them in your data structure
* You can now sample such transitions in proportion to the error (see [article](https://arxiv.org/abs/1511.05952)) for training.

It's probably more convenient to explicitly declare inputs for "sample observations", "sample actions" and so on to plug them into q-learning.

Prioritized (and even normal) experience replay should greatly reduce amount of game sessions you need to play in order to achieve good performance. 

While it's effect on runtime is limited for atari, more complicated envs (further in the course) will certainly benefit for it.

There is even more out there - see this [overview article](https://arxiv.org/abs/1710.02298).

In [ ]:
session_rewards = np.array(sessions)

In [ ]:
from submit import submit_breakout
EMAIL = ''
TOKEN = ''

submit_breakout(session_rewards, EMAIL, TOKEN)